In [1]:
from imports.TactileDataset import TactileDataset

In [2]:
td = TactileDataset(root="/home/hussain/tactile/data/contact_extraction2/train/")
td

TactileDataset(407)

In [3]:
len(samples)

NameError: name 'samples' is not defined

In [ ]:
c = [v['case'] for k,v in samples.items()]
n_ev = [len(v['events']) for k,v in samples.items()]

In [6]:
def load_all_raw(root):
    samples = {}
    for subset in ['train', 'val', 'test']:
        with open(root / subset / 'raw' / 'contact_cases.json', 'r') as f:
            import json
            subset_samples = json.load(f)
            subset_samples_tot_idx = {item[1]['total_idx']: 
                                      {
                                          'events': item[1]['events'], 
                                          'case': item[1]['case']
                                      } for item in subset_samples.items()}
        
        samples.update(subset_samples_tot_idx)
    return samples
from pathlib import Path          
samples = load_all_raw(Path("/home/hussain/tactile/data/contact_extraction2/"))

In [1]:
with open('/home/hussain/tactile/data/contact_extraction1/raw/case.pkl', 'rb') as f:
    import pickle
    c = pickle.load(f)

In [4]:
plt.hist(n_ev)

NameError: name 'plt' is not defined

In [5]:
import numpy as np
import matplotlib.pyplot as plt

cases, count = np.unique(c, return_counts=True)
plt.bar(cases, count)

NameError: name 'c' is not defined

In [11]:
sum(count)

686

In [8]:
possible_angle = [0.0174532925, 0.034906585, 0.0523598776, 0.075, 0.095, 0.115, 0.135, 0.15]#
N_examples = 9
list_of_rotations = [[0, 0, 0]]

for i in range(1, N_examples):
    theta = i * 2 * np.pi/(N_examples - 1)
    for phi in possible_angle:
        rx = phi * np.cos(theta)
        ry = phi * np.sin(theta)
        rotvec = [rx, ry, 0]
        list_of_rotations.append(rotvec)

print(len(list_of_rotations))
print(list_of_rotations)


65
[[0, 0, 0], [0.01234134148078231, 0.012341341480782309, 0], [0.02468268296156462, 0.024682682961564617, 0], [0.03702402451305761, 0.037024024513057606, 0], [0.053033008588991064, 0.05303300858899106, 0], [0.06717514421272203, 0.06717514421272201, 0], [0.08131727983645297, 0.08131727983645295, 0], [0.09545941546018392, 0.09545941546018391, 0], [0.10606601717798213, 0.10606601717798211, 0], [1.0687059397353753e-18, 0.0174532925, 0], [2.1374118794707506e-18, 0.034906585, 0], [3.2061178253293598e-18, 0.0523598776, 0], [4.592425496802574e-18, 0.075, 0], [5.817072295949928e-18, 0.095, 0], [7.04171909509728e-18, 0.115, 0], [8.266365894244634e-18, 0.135, 0], [9.184850993605149e-18, 0.15, 0], [-0.012341341480782309, 0.01234134148078231, 0], [-0.024682682961564617, 0.02468268296156462, 0], [-0.037024024513057606, 0.03702402451305761, 0], [-0.05303300858899106, 0.053033008588991064, 0], [-0.06717514421272201, 0.06717514421272203, 0], [-0.08131727983645295, 0.08131727983645297, 0], [-0.09545941

In [5]:
import torch
from torch.utils.data.dataset import Subset
import torch.nn as nn 
from torch.nn import Sequential, Linear, ReLU
#from torch import autograd1
from torch.autograd import Variable
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.utils import normalized_cut
from torch_geometric.nn import voxel_grid, max_pool, max_pool_x, graclus, global_mean_pool, GCNConv,  global_mean_pool
#from torch_geometric.nn.conv import SplineConv
#from torch_geometric.nn.norm import BatchNorm
from torch_geometric.data import InMemoryDataset, download_url
from collections.abc import Sequence
from torch_geometric.nn import global_mean_pool
import time

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GCNConv(1, 64)
        self.bn1 = torch.nn.BatchNorm1d(64)

        self.conv2 = GCNConv(64, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        
        self.conv3 = GCNConv(64, 512)
        self.bn3 = torch.nn.BatchNorm1d(512) 
        
        self.conv4 = GCNConv(512, 256)
        self.bn4 = torch.nn.BatchNorm1d(256)  
        
        self.conv5 = GCNConv(256, 64)
        self.bn5 = torch.nn.BatchNorm1d(64)
        
        self.conv6 = GCNConv(64, 32)
        self.bn6 = torch.nn.BatchNorm1d(32)

        self.conv7 = GCNConv(32, 16)
        self.bn7 = torch.nn.BatchNorm1d(16)        

        self.conv8 = GCNConv(16, 2)
        self.bn8 = torch.nn.BatchNorm1d(16)    
        

        self.fc1 = torch.nn.Linear(16, 16)
        self.fc2 = torch.nn.Linear(16, 2)
        


    def forward(self, data):
        #data.x = F.elu(self.conv1(data.x , data.edge_index , data.edge_attr ))
        # print("index", data.x.view(-1,2).shape)
        data.x = F.sigmoid(self.conv1(torch.tensor(data.x.view(-1,1), dtype=torch.float32), data.edge_index) )#, data.edge_attr ))
        data.x = self.bn1(data.x)


        data.x = F.sigmoid(self.conv2(data.x, data.edge_index))
        data.x = self.bn2(data.x)


        data.x = F.sigmoid(self.conv3(data.x, data.edge_index))
        data.x = self.bn3(data.x)

        data.x = F.sigmoid(self.conv4(data.x, data.edge_index))
        data.x = self.bn4(data.x)

        data.x = F.sigmoid(self.conv5(data.x, data.edge_index))
        data.x = self.bn5(data.x)

        data.x = F.sigmoid(self.conv6(data.x, data.edge_index))
        data.x = self.bn6(data.x)

        data.x = F.sigmoid(self.conv7(data.x, data.edge_index))
        data.x = self.bn7(data.x)

        # data.x = F.sigmoid(self.conv8(data.x, data.edge_index))
                # 2. Readout layer
        x = global_mean_pool(data.x, data.batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.sigmoid(self.fc1(x))
        #print("abdulrahman2 done",x.size())

        out = F.sigmoid(self.fc2(x))
        
        # print("out", out)
#         out=F.softmax(data.x, dim=1)
        return out

model=Net()

In [ ]:
from tqdm.auto import tqdm
train_loader = DataLoader(td, batch_size=1, shuffle=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device", device)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
i=0
iv=0
epoch_losses = []
epoch_lossesv = []
number_of_epoch=20
print ("Training will start now")
for epoch in range(number_of_epoch):
    print("Epoch", epoch)
    epoch_loss = 0
    acc=0
    start=time.time()    

    for i, data in enumerate(tqdm(train_loader)):
        with torch.autograd.detect_anomaly():
            data = data.to(device)
            #print(data.y)
            optimizer.zero_grad()
            end_point = model(data) 

            loss = nn.L1Loss()(end_point.view(-1,2)[0], data.y.view(-1,2)[0])
            pred = end_point.max(1)[1]
            acc += (pred.eq(data.y).sum().item())/len(data.y)

            loss.backward()
            optimizer.step()
            # print("loss",loss)
            # if i % 10 == 0:
            #     print({'epoch': epoch,'batch': i + 1,'loss': loss.item(),'acc': acc})

            epoch_loss += loss.detach().item()
            i=i+1
    epoch_loss /= (i + 1)
    acc /= (i + 1)

    epoch_lossv = 0


    #for iv, datav in enumerate(test_loader):
# print(data)
#         with autograd.detect_anomaly():
        #datav = datav.to(device)
        #print(data.y)
        #end_pointv = model(datav)
        #print("bbbbb end_point",(end_point))    
        # print("bbbbb data.y",(data.y.view(-1,2) ))  
        # print("bbbbb end_point",(end_point.view(-1,2)[0] ))  

        #lossv = nn.L1Loss()(end_pointv.view(-1,2)[0], datav.y.view(-1,2)[0])
        

        # print("loss",loss)
        # if i % 10 == 0:
        #     print({'epoch': epoch,'batch': i + 1,'loss': loss.item(),'acc': acc})
        
        #epoch_lossv += lossv.detach().item()
        #iv=iv+1
        #pass
    #epoch_lossv /= (i + 1)



    end = time.time()
    print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss), ' Elapsed time: ', end-start)
    epoch_losses.append(epoch_loss)
    #epoch_lossesv.append(epoch_lossv)


device cuda:0
Training will start now
Epoch 0


  0%|                                           | 0/686 [00:00<?, ?it/s]/tmp/ipykernel_28970/2442399191.py:21: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with torch.autograd.detect_anomaly():
/tmp/ipykernel_28970/2024586760.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.x = F.sigmoid(self.conv1(torch.tensor(data.x.view(-1,1), dtype=torch.float32), data.edge_index) )#, data.edge_attr ))
100%|█████████████████████████████████| 686/686 [01:52<00:00,  6.08it/s]


Epoch 0, loss 0.0517  Elapsed time:  112.82031679153442
Epoch 1


100%|█████████████████████████████████| 686/686 [01:53<00:00,  6.07it/s]


Epoch 1, loss 0.0516  Elapsed time:  113.00654673576355
Epoch 2


...

100%|█████████████████████████████████| 686/686 [01:52<00:00,  6.07it/s]


Epoch 11, loss 0.0516  Elapsed time:  112.94505643844604
Epoch 12


100%|█████████████████████████████████| 686/686 [01:52<00:00,  6.09it/s]


Epoch 12, loss 0.0516  Elapsed time:  112.70003461837769
Epoch 13


 57%|██████████████████▉              | 393/686 [01:04<00:51,  5.65it/s]